<a href="https://colab.research.google.com/github/SOKAJ-7/Module-16-Cloud-ETL/blob/main/Vine_reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [7]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with total votes equal to or greater than 20.
cleaned_vine_df = vine_df.filter(col("total_votes") >= 20)
cleaned_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [8]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
import pyspark.sql.functions as F
cleaned_total_df = cleaned_vine_df.withColumn('result_column',((F.col("helpful_votes") / F.col("total_votes"))*100)).filter('result_column>50')
cleaned_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|61.76470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|             60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|             84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|91.66666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|78.46153846153847|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|86.11111111111111|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|55.88235294117647|
|R36O341WWXXKNP|          5|           2

In [9]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_paid_df = cleaned_total_df.filter(col("vine") == "Y")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|88.88888888888889|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|            100.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|79.16666666666666|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|84.61538461538461|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|76.92307692307693|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|90.19607843137256|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|78.57142857142857|
| RQ5WD90PUNBU9|          5|           2

In [10]:
# retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_unpaid_df = cleaned_total_df.filter(col("vine") == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|61.76470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|             60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|             84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|91.66666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|78.46153846153847|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|86.11111111111111|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|55.88235294117647|
|R36O341WWXXKNP|          5|           2

In [11]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid)
total_review_paid = vine_paid_df.select("vine").count()
total_review_paid

94

In [12]:
five_star_paid = vine_paid_df.filter("star_rating == 5").count()
five_star_paid

48

In [13]:
percent_five_star_paid = (five_star_paid / total_review_paid) * 100
percent_five_star_paid

51.06382978723404

In [14]:
total_review_unpaid = vine_unpaid_df.select("vine").count()
total_review_unpaid

39915

In [15]:
five_star_unpaid = vine_unpaid_df.filter("star_rating == 5").count()
five_star_unpaid

15556

In [16]:
percent_five_star_unpaid = (five_star_unpaid / total_review_unpaid) * 100
percent_five_star_unpaid

38.97281723662783